In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from datetime import date
from bs4 import BeautifulSoup
import sys
import time
import pandas as pd
import datetime
import urllib.request

now = datetime.datetime.now().strftime("%Y%m%d")

In [17]:
#new chrome driver
def OpenChromeDriver():
    #開啟瀏覽器新分頁，下載google chrome driver
    option = webdriver.ChromeOptions()

    #fix chrome 103 problem
    option.binary_location = "/Applications/Google Chrome Beta.app/Contents/MacOS/Google Chrome Beta"

    prefs={"profile.default_content_setting_values.notifications" : 2}
    option.add_experimental_option("prefs",prefs)
    gchrome=webdriver.Chrome(options=option)
    return gchrome

def FinishedCrawler(driver):
    driver.close()

In [31]:
#return name and download icon

def getPhoneDetailData(brand_name, soup):
    #param
    phone_list =[]

    phones = soup.find_all("div", class_="flex_gadget_wrap")
    if(phones):
        for phone in phones:
            phone_img_div = phone.find("div", class_="image")
            phone_img = phone_img_div.find("img")["src"]

            phone_data_div = phone.find("div", class_="model_no")
            phone_data_a = phone_data_div.find("a")
            phone_name = phone_data_a.get_text()
            if(phone_name.find("/")>-1):
                phone_name = phone_name.replace("/","_")
            phone_link = "https://www.dcfever.com"+phone_data_a["href"]
            img_save_name = "../icon/"+phone_name+".jpg"
            time.sleep(0.5)
            urllib.request.urlretrieve(phone_img, img_save_name)

            phone_list.append([brand_name, phone_name, phone_link, img_save_name])
            
    return phone_list

In [19]:
#return phone of a brand

def getBrandPhoneData(brand_name, url, driver):
    phone_list =[]
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    pages_div = soup.find("div", class_="pages")
    pages = pages_div.find_all("a")[:-1]

    #先加入第一頁
    phone_list+=getPhoneDetailData(brand_name, soup)

    if pages:
        for page in pages:
            temp_link = "https://www.dcfever.com/"+page["href"]
            driver.get(temp_link)
            time.sleep(0.5)
            soup = BeautifulSoup(driver.page_source, "html.parser")
            phone_list+=getPhoneDetailData(brand_name, soup)
    
    else:
        phone_list+=getPhoneDetailData(brand_name, soup)

    return phone_list

In [20]:
#return brands

def getBrandData(url, driver):

    #param
    brand_list = []
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    brand_matrix = soup.find("ul", class_="brand_matrix clearfix")
    
    for brand in brand_matrix.find_all("li"):

        #brand name
        temp_brand_name_link = brand.find("img")["src"]
        temp_brand_name = temp_brand_name_link.replace("https://cdn09.dcfever.com/media/brands/","")
        temp_brand_name = temp_brand_name.replace(".gif","")
        temp_brand_name = temp_brand_name.replace(".jpg","")

        #brand link
        temp_brand_link = "https://www.dcfever.com/phones/"+brand.find("a")["href"]
        brand_list.append([temp_brand_name, temp_brand_link])

    #save brand
    brand_df = pd.DataFrame(data=brand_list, columns=['Brand_Name', 'Link'])
    file_name = "../crawler_data/Phone_Brand_Data"+now+".csv"
    brand_df.to_csv(file_name, index=False, encoding="utf-8")

    return True

if(getBrandData("https://www.dcfever.com/phones/database.php", OpenChromeDriver())):
    print("done")
else:
    print("fail")

In [14]:
brand_df = pd.read_csv("../crawler_data/Phone_Brand_Data20220719.csv")
brand_df

,Brand_Name,Link
0,apple,https://www.dcfever.com/phones/brand.php?id=49
1,asus,https://www.dcfever.com/phones/brand.php?id=50
2,google,https://www.dcfever.com/phones/brand.php?id=173
3,honorlogo,https://www.dcfever.com/phones/brand.php?id=312
4,htc,https://www.dcfever.com/phones/brand.php?id=61
5,huawei_logo,https://www.dcfever.com/phones/brand.php?id=151
6,microsoft,https://www.dcfever.com/phones/brand.php?id=141
7,motorola,https://www.dcfever.com/phones/brand.php?id=66
8,nokia,https://www.dcfever.com/phones/brand.php?id=68
9,Nothing_Logo.webp.png,https://www.dcfever.com/phones/brand.php?id=328


In [32]:
#param
phone_list = []
target = brand_df.loc[brand_df['Brand_Name'] == "sharp"]

phone_list+= getBrandPhoneData(target['Brand_Name'].values[0], target['Link'].values[0], OpenChromeDriver())
time.sleep(1)

print("done")

done


In [33]:
sharp_phone_df = pd.DataFrame(data=phone_list, columns=['Brand_Name', 'Phone_Name', 'Link', 'Icon'])
sharp_phone_df

,Brand_Name,Phone_Name,Link,Icon
0,sharp,Sharp Aquos R6,https://www.dcfever.com/phones/specification.p...,../icon/Sharp Aquos R6.jpg
1,sharp,Sharp Aquos R7,https://www.dcfever.com/phones/specification.p...,../icon/Sharp Aquos R7.jpg
2,sharp,Sharp AQUOS S2,https://www.dcfever.com/phones/specification.p...,../icon/Sharp AQUOS S2.jpg
3,sharp,Sharp SH930W,https://www.dcfever.com/phones/specification.p...,../icon/Sharp SH930W.jpg
4,sharp,Sharp SH837W,https://www.dcfever.com/phones/specification.p...,../icon/Sharp SH837W.jpg
5,sharp,Sharp SH631W,https://www.dcfever.com/phones/specification.p...,../icon/Sharp SH631W.jpg
6,sharp,Sharp SH530U,https://www.dcfever.com/phones/specification.p...,../icon/Sharp SH530U.jpg
7,sharp,Sharp Aquos SH-01D,https://www.dcfever.com/phones/specification.p...,../icon/Sharp Aquos SH-01D.jpg
8,sharp,Sharp docomo SH-10C,https://www.dcfever.com/phones/specification.p...,../icon/Sharp docomo SH-10C.jpg
9,sharp,Sharp docomo SH-11C,https://www.dcfever.com/phones/specification.p...,../icon/Sharp docomo SH-11C.jpg


In [34]:
phone_df = pd.read_csv("../crawler_data/Phone_Data_20220802.csv")
phone_df

,Brand_Name,Phone_Name,Link,Icon
0,apple,iPhone SE 3,https://www.dcfever.com/phones/specification.p...,../icon/iPhone SE 3.jpg
1,apple,iPhone 13 mini,https://www.dcfever.com/phones/specification.p...,../icon/iPhone 13 mini.jpg
2,apple,iPhone 13,https://www.dcfever.com/phones/specification.p...,../icon/iPhone 13.jpg
3,apple,iPhone 13 Pro,https://www.dcfever.com/phones/specification.p...,../icon/iPhone 13 Pro.jpg
4,apple,iPhone 13 Pro Max,https://www.dcfever.com/phones/specification.p...,../icon/iPhone 13 Pro Max.jpg
...,...,...,...,...
1166,miui,小米 3,https://www.dcfever.com/phones/specification.p...,../icon/小米 3.jpg
1167,miui,MIUI 小米 2,https://www.dcfever.com/phones/specification.p...,../icon/MIUI 小米 2.jpg
1168,miui,MIUI 小米 2S,https://www.dcfever.com/phones/specification.p...,../icon/MIUI 小米 2S.jpg
1169,miui,紅米 Note 5,https://www.dcfever.com/phones/specification.p...,../icon/紅米 Note 5.jpg


In [35]:
new_phone_df = pd.concat([phone_df, sharp_phone_df],axis=0, ignore_index=True)
new_phone_df

,Brand_Name,Phone_Name,Link,Icon
0,apple,iPhone SE 3,https://www.dcfever.com/phones/specification.p...,../icon/iPhone SE 3.jpg
1,apple,iPhone 13 mini,https://www.dcfever.com/phones/specification.p...,../icon/iPhone 13 mini.jpg
2,apple,iPhone 13,https://www.dcfever.com/phones/specification.p...,../icon/iPhone 13.jpg
3,apple,iPhone 13 Pro,https://www.dcfever.com/phones/specification.p...,../icon/iPhone 13 Pro.jpg
4,apple,iPhone 13 Pro Max,https://www.dcfever.com/phones/specification.p...,../icon/iPhone 13 Pro Max.jpg
...,...,...,...,...
1222,sharp,Sharp SX633A,https://www.dcfever.com/phones/specification.p...,../icon/Sharp SX633A.jpg
1223,sharp,Sharp 102SH,https://www.dcfever.com/phones/specification.p...,../icon/Sharp 102SH.jpg
1224,sharp,Sharp 104SH,https://www.dcfever.com/phones/specification.p...,../icon/Sharp 104SH.jpg
1225,sharp,Sharp 816SH,https://www.dcfever.com/phones/specification.p...,../icon/Sharp 816SH.jpg


In [36]:
#save phone
new_phone_df['Brand_Name'] = new_phone_df['Brand_Name'].replace(['honorlogo'], "honor")
new_phone_df['Brand_Name'] = new_phone_df['Brand_Name'].replace(['huawei_logo'], "huawei")
new_phone_df['Brand_Name'] = new_phone_df['Brand_Name'].replace(['Nothing_Logo.webp.png'], "Nothing")
new_phone_df['Brand_Name'] = new_phone_df['Brand_Name'].replace(['oppologo'], "oppo")
new_phone_df['Brand_Name'] = new_phone_df['Brand_Name'].replace(['realne'], "realme")
new_phone_df.drop_duplicates(subset ="Phone_Name", keep = "first", inplace = True)

file_name = "../crawler_data/Phone_Data_"+now+".csv"
new_phone_df.to_csv(file_name, index=False, encoding="utf-8")

In [37]:
new_phone_df = pd.read_csv("../crawler_data/Phone_Data_20220809.csv")
new_phone_df

,Brand_Name,Phone_Name,Link,Icon
0,apple,iPhone SE 3,https://www.dcfever.com/phones/specification.p...,../icon/iPhone SE 3.jpg
1,apple,iPhone 13 mini,https://www.dcfever.com/phones/specification.p...,../icon/iPhone 13 mini.jpg
2,apple,iPhone 13,https://www.dcfever.com/phones/specification.p...,../icon/iPhone 13.jpg
3,apple,iPhone 13 Pro,https://www.dcfever.com/phones/specification.p...,../icon/iPhone 13 Pro.jpg
4,apple,iPhone 13 Pro Max,https://www.dcfever.com/phones/specification.p...,../icon/iPhone 13 Pro Max.jpg
...,...,...,...,...
1088,sharp,Sharp SX633A,https://www.dcfever.com/phones/specification.p...,../icon/Sharp SX633A.jpg
1089,sharp,Sharp 102SH,https://www.dcfever.com/phones/specification.p...,../icon/Sharp 102SH.jpg
1090,sharp,Sharp 104SH,https://www.dcfever.com/phones/specification.p...,../icon/Sharp 104SH.jpg
1091,sharp,Sharp 816SH,https://www.dcfever.com/phones/specification.p...,../icon/Sharp 816SH.jpg
